In [4]:
import openai
import json
import pprint
import os
import random

with open('params.json') as f:
    params = json.load(f)

openai.api_key = params['OPENAI_API_KEY']
openai.organization = params['OPENAI_ORG']

def get_response(prompt, model='gpt-3.5-turbo'):
    result = openai.ChatCompletion.create(
    model=model,
    messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
    ])

    return result.choices[0]['message']['content']

### Locations

1. University Classes
2. Mr Hi's karate studio
3. University Rathskeller
4. Bar
5. Open Karate Tournaments (held at the private karate studios)
6. Intercollegiate Karate Tournament (held at local universities)

Locations 5 and 6 are mutually exclusive

### Network Formation

Edge creation process:
1.  Association in and between academic classes at the university.
2. Membership in John Doe's private karate studio on the east sid the city where John Doe taught nights as a part-time instructor.
3. Membership in John Doe's private karate studio on the east sid the city, where many of his supporters worked out on weekends.
4. Student teaching at the east-side karate studio referred to in This is different from (2) in that student teachers interacted with other, but were prohibited from interacting with their students.
5. Interaction at the university rathskeller, located in the s basement as the karate club's workout area.
6. Interaction at a student-oriented bar located across the street from the university campus.
7. Attendance at open karate tournaments held through the area at private karate studios.
8. Attendance at intercollegiate karate tournaments held at local universities. Since both open and intercollegiate tournaments were held on Saturdays, attendance at both was impossible.



In [5]:
# TODO:
# 1. Generate a list of students without prompt, examine the network structure. (GPTTeam generates a database.)
# Network sturcture: clusters, power law, ties spanning two different classes


def generate_karate_club_locations(num_classes=3):

    locations_prompt = f"""
    # Task 
    You are an AI assistant and your task is to generate a short description of some locations regarding a university. 
    The locations are the following: 
    * {num_classes} academic classes at the university. The classes can be from a diverse set of subjects, such as math, physics, chemistry, biology, computer science, psychology, law, journalism, policital scienceetc.
    * A karate studio belonging to John Doe.
    * A bar located accross the street from the university campus.
    * A university rathskeller.
    * A dojo for holding open karate tournaments through the area at private karate studios.
    * A dojo for intercollegiate karate tournaments belonging to the university.


    # Output
    You must format your output as a list of JSON objects with the following fields:

    [
        {{
            "name" : name of location,
            "description" : description of location,
            "type" : type of location (e.g. academic class, karate studio, bar, etc.)
        }},
        ...
    ]

    ## Notes
    * You must output JSON objects for each location.
    
    ```json
    """

    response = get_response(locations_prompt).split('```')[0]

    with open('outputs/locations.json', 'w+') as f:
        f.write(response)

    return json.loads(response)

def is_in_case(x, y):
    x = x.lower()
    return x in y or x.upper() in y or x.capitalize() in y

def generate_karate_club_agents(locations, num_agents=3):

    for location in locations:
        if location['type'] == 'karate studio':
            karate_studio = location['name']
            break
    
    # Karate instructors
    agents = [
        {
            'first_name' : 'John Doe',
            'private_bio' : """John Doe is the karate instructor at the karate studio.
                            John Doe wants to raise prices, claiming the authority to set his own lesson fees, since he is the instructor.""",
            'public_bio' : """John Doe is the karate instructor at the karate studio.
                            John Doe wants to raise prices, claiming the authority to set his own lesson fees, since he is the instructor.""",
            'directives' : [
                'Teach karate.',
                'Discuss about the lesson fees.'
                'Be consise and to the point. Avoid small talk.',
            ],
            'initial_plan' : {
                'location' : karate_studio,
                'stop_condition' : 'Get as many students as possible on his side.',
                'description' : 'John Doe is at the karate studio. There is a conflict between John Doe and Richard Roe regarding the lesson fees.'
            }
        },
        {
            'first_name' : 'Richard Roe',
            'private_bio' : 'Richard Roe is the chief administrator of the karate club. Richard roe wants to stabilize the lesson fees, claiming his authority as the chief administrator.',
            'public_bio' : 'Richard Roe is the chief administrator of the karate club. Richard roe wants to stabilize the lesson fees, claiming his authority as the chief administrator.',
            'directives' : [
                'Manage the karate club.',
                'Discuss about the lesson fees.'
                'Be consise and to the point. Avoid small talk.',
            ],
            'initial_plan' : {
                'location' : karate_studio,
                'stop_condition' : 'Get as many students as possible on his side.',
                'description' : 'Richard Roe is at the karate studio. There is a conflict between John Doe and Richard Roe regarding the lesson fees.'
            }
        }
    ]


    num_students = num_agents - 2

    # Students
    for i in range(num_students):
        # agent_location = random.choice(locations)['name']
        agent = {
            'first_name' : f'Student {i + 1}',
            'private_bio' : f'Student {i + 1} a student at the university.',
            'public_bio' : f'Student {i + 1} is a student at the university.',
            'directives' : [
                'Make friends.',
                'Respond to other students, Jonh Doe, and Richard Roe.',
                'Express opinions about the lesson fees.'
                'Attend classes.',
                'Study.',
                'Play karate.',
                'Be consise and to the point..'
            ],
            'initial_plan' : {
                'location' : karate_studio,
                'stop_condition' : 'Decide to support either John Doe or Richard Roe.',
                'description' : f'Student {i + 1} is at {karate_studio}.'
            }
        }


        agents.append(agent)

    # with open('outputs/agents.json', 'w+') as f:
    #     f.write(json.dumps(agents, indent=4))

    return agents

def create_gptteam_world(num_classes=3, replace=True):

    if os.path.exists('outputs/locations.json'):
        with open('outputs/locations.json') as f:
            locations = json.load(f)
    else:
        locations = generate_karate_club_locations(num_classes=num_classes)

    if os.path.exists('outputs/agents.json'):
        with open('outputs/agents.json') as f:
            agents = json.load(f)
    else:
        agents = generate_karate_club_agents(locations=locations)

    config = {
        "world_name" : "University of Anytown",
        "locations" : locations,
        "agents" : agents
    }

    with open('outputs/config.json', 'w+') as f:
        f.write(json.dumps(config, indent=4))

    if replace:
        with open('../GPTeam/config.json', 'w+') as f:
            f.write(json.dumps(config, indent=4))


In [6]:
create_gptteam_world()